In [ ]:
import pandas as pd
import json
import os
import numpy as np

import shapely


In [6]:
data=pd.read_json('static/Minneapolis_Neighborhoods.geojson')

data=data.features

i = 0
neighborhoods=[]

for items in data:
    dict={}
    dict['neighborhood']=data[i]['properties']['BDNAME']
    dict['geometry'] = data[i]['geometry']['coordinates']
    neighborhoods.append(dict)
    i+=1

neighborhoods_df=pd.DataFrame(neighborhoods)

neighborhoods_df.head()

,neighborhood,geometry
0,Phillips West,"[[[-93.2625807586419, 44.9609082137146], [-93...."
1,Downtown West,"[[[-93.2601055025157, 44.9829952758614], [-93...."
2,Downtown East,"[[[-93.2449864570206, 44.9789336625517], [-93...."
3,Ventura Village,"[[[-93.2495770034483, 44.9662967560422], [-93...."
4,Sumner - Glenwood,"[[[-93.2882976528817, 44.9890356035354], [-93...."


In [8]:
yelp_data=pd.read_json('http://127.0.0.1:5000/yelp_data')
yelp_data

,address,categories,image,index,latitude,longitude,name,phone,rating,reviews,transactions,url,yelpid
0,"1121 Hennepin Ave Minneapolis, MN 55403","[American (New), Beer Gardens, Venues & Event ...",https://s3-media1.fl.yelpcdn.com/bphoto/raQA0N...,0,44.974777,-93.279792,Butcher & The Boar,(612) 238-8888,4.5,1615,[delivery],https://www.yelp.com/biz/butcher-and-the-boar-...,G9KdODINirmdSuYaBfBppA
1,"800 Washington Ave N Minneapolis, MN 55401","[Italian, Desserts, Wine Bars]",https://s3-media2.fl.yelpcdn.com/bphoto/SKbCky...,1,44.989540,-93.278594,Bar La Grassa,(612) 333-3837,4.5,1251,[],https://www.yelp.com/biz/bar-la-grassa-minneap...,zv_XJAQr9D3PNR5eHYhC5w
2,"112 N 3rd St Minneapolis, MN 55401","[American (New), Wine Bars, Venues & Event Spa...",https://s3-media2.fl.yelpcdn.com/bphoto/6oZdXE...,2,44.982670,-93.271600,112 Eatery,(612) 343-7696,4.5,1166,[delivery],https://www.yelp.com/biz/112-eatery-minneapoli...,6vaAze9Fxuco249L3BhEdQ
3,"813 W 50th St Minneapolis, MN 55419","[American (New), Pubs, British]",https://s3-media3.fl.yelpcdn.com/bphoto/u7Pn5-...,3,44.912151,-93.290436,George & The Dragon,(612) 208-1047,4.5,889,"[delivery, pickup]",https://www.yelp.com/biz/george-and-the-dragon...,5s2sT8ux7og5XfK-180r2g
4,"211 N 1st St Minneapolis, MN 55401","[American (New), Bars]",https://s3-media2.fl.yelpcdn.com/bphoto/QaDSlg...,4,44.985489,-93.269536,Spoon and Stable,(612) 224-9850,4.5,893,[delivery],https://www.yelp.com/biz/spoon-and-stable-minn...,5olLs-K-_k_rWrrNLjvy0w
...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,"200 S 6th St Minneapolis, MN 55402",[Cafes],https://s3-media1.fl.yelpcdn.com/bphoto/kX6ZMF...,813,44.977082,-93.267680,Caribou Coffee,(612) 338-1170,3.5,13,[delivery],https://www.yelp.com/biz/caribou-coffee-minnea...,ulhaJ_sD5C8Tp2HrTOTO3g
813,"430 1st Ave N Ste 035 Minneapolis, MN 55401","[Dance Clubs, Sports Bars, American (Tradition...",https://s3-media4.fl.yelpcdn.com/bphoto/037uvX...,814,44.980910,-93.273720,Brothers Bar & Grill,(612) 339-4240,2.5,80,[],https://www.yelp.com/biz/brothers-bar-and-gril...,96GabBtK6mWKGD1tbHKgdA
814,"312 W Lake St Minneapolis, MN 55408","[Mediterranean, American (New), Greek]",https://s3-media3.fl.yelpcdn.com/bphoto/IRxolE...,815,44.949151,-93.283917,New York Gyro,(612) 353-5550,4.0,10,"[pickup, delivery]",https://www.yelp.com/biz/new-york-gyro-minneap...,LvGXa3_xmSqOoGS3fsPG5g
815,"3404 Nicollet Ave S Minneapolis, MN 55408","[Chicken Wings, Fish & Chips]",https://s3-media1.fl.yelpcdn.com/bphoto/V9wsod...,816,44.941131,-93.277985,Wings and Seafood To Go,(612) 825-8682,3.5,17,[],https://www.yelp.com/biz/wings-and-seafood-to-...,Z7FQJU7hq7OGhTN6ReNsIA


In [9]:
i=0
yelp_list=[]

for row in yelp_data:
    pointLon = yelp_data['longitude']
    pointLat = yelp_data['latitude']
    point=list(pointLon,pointLat)
    yelp_list.append(point)

yelp_list
    
    

TypeError: list() takes at most 1 argument (2 given)